In [88]:
from csv import DictReader
import numpy as np
#Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary

podatki = []
dtype = [
    ('Date', 'U10'),           # Date as string
    ('Time', 'U10'),            # Time as string
    ('Location', 'U100'),       # Location as string
    ('Operator', 'U100'),       # Operator as string
    ('Flight', 'U100'),         # Flight number as string
    ('Route', 'U100'),          # Route as string
    ('Type', 'U100'),           # Aircraft type as string
    ('Registration', 'U100'),   # Registration as string
    ('cn_In', 'U100'),          # Construction number or other identifier as string
    ('Aboard', np.int32),           # Number of people aboard as integer
    ('Fatalities', np.int32),       # Number of fatalities as integer
    ('Ground', np.int32),           # Number of ground fatalities as integer
    ('Summary', 'U1000')        # Summary as string
]

with open("podatki\Airplane_Crashes_and_Fatalities_Since_1908.csv", 'r', encoding='utf-8') as file:
    csv_reader = DictReader(file)
    for row in csv_reader:
        date = row['Date']
        time = row['Time']
        location = row['Location']
        operator = row['Operator']
        flightNr = row['Flight #']
        route = row['Route']
        type = row['Type']
        registration = row['Registration']
        CnIn = row['cn/In']
        aboard = int(row["Aboard"]) if row["Aboard"] else 0
        fatalities = int(row["Fatalities"]) if row["Fatalities"] else 0
        ground = int(row['Ground']) if row["Ground"] else 0
        summary = row['Summary']
        podatki.append((date,time,location,operator,flightNr,route,type,registration,CnIn,aboard,fatalities,ground,summary))
podatki = np.sort(np.array(podatki,dtype=dtype), order='Date')




#print(podatki)



***Uničena***

In [89]:
def unicena_letala():
    unicena = np.array([entry for entry in podatki if (
                                                        "shoot" in entry["Summary"].lower() 
                                                        or "shot" in entry["Summary"].lower()
                                                        or "attack" in entry["Summary"].lower()
                                                    )
                                                    and not(
                                                        "overshoot" in entry["Summary"].lower()
                                                        or "undershoot" in entry["Summary"].lower()
                                                    )
                                                    ])
    print(len(unicena))
    years,counts = np.unique([entry['Date'][-4:] for entry in unicena],return_counts=True)
    for year,count in zip(years,counts):
        if(count>3):
            print(year,count)
unicena_letala()

209
1940 5
1942 7
1943 4
1944 8
1952 4
1964 4
1966 4
1968 4
1969 4
1970 4
1972 9
1980 5
1986 5
1987 7
1988 8
1990 6
1991 8
1994 7
1996 5
1998 4
2007 4


#

***Faulty design***

In [90]:
def faulty_letala():
    faulty = np.array([entry for entry in podatki if (
                                                        "faulty design" in entry["Summary"].lower() 
                                                        or "design flaw" in entry["Summary"].lower()
                                                        or "design deficiencies" in entry["Summary"].lower()

                                                    )

                                                    ])
    print(len(faulty["Summary"]))

faulty_letala()

16


***Water***

In [91]:
def water_letala():
    faulty = np.array([entry for entry in podatki if (
                                                        "ocean" in entry["Summary"].lower() 
                                                        or "water" in entry["Summary"].lower()
                                                        or "swamp" in entry["Summary"].lower()
                                                        or "sea" in entry["Summary"].lower()    
                                                        or "river" in entry["Summary"].lower()  
                                                    )

                                                    ])
    print(len(faulty["Summary"]))

water_letala()

617


***Low fuel***

In [92]:
def fuel_letala():
    faulty = np.array([entry for entry in podatki if (
                                                        "low fuel" in entry["Summary"].lower() 
                                                        or "fuel exhaustion" in entry["Summary"].lower() 
                                                        or "ran out of fuel" in entry["Summary"].lower() 
                                                        or "running out of fuel" in entry["Summary"].lower() 
                                                    )

                                                    ])
    print(len(faulty["Summary"]))

fuel_letala()

66


***Birds***

In [93]:
def bird_letala():
    faulty = np.array([entry for entry in podatki if (
                                                        "bird" in entry["Summary"].lower()
                                                        or "geese" in entry["Summary"].lower()
                                                        or "flock" in entry["Summary"].lower()
                                                    )

                                                    ])
    print(len(faulty["Summary"]))
    print(faulty["Summary"])

bird_letala()

12
['The plane was taking off from La Guardia Airport when possibly, both engines were disabled after striking a flock of geese. The crew was able to ditch the plane in the Hudson River, making a soft landing. All 150 passengers and crew of 5 made it out safely before the plane began to sink. No one was reported to be seriously injured.'
 'The plane flew into a flock of birds that were injested into the left engine.  The pilot soon after reported an engine fire and that he was returning to the airport.  However, control of the plane was lost and the aircraft crashed.'
 'The crew abandoned their takeoff after the loss of an engine due to bird ingestion. The jet overshot the runway and collided with a truck, killing the driver. The landing gear sheared off and the plane came to rest in a field.'
 "The aircraft was believed to have gone into a spin or spiral dive with recovery at about 12,000 ft. The dive apparently caused structural damage which resulted in the aircraft plunging into St 

In [116]:
import re
from geopy.geocoders import Nominatim



import multiprocessing



def geocode_worker(address):
    address.geocode()
    return address


#print(podatki["Location"])

geolocator = Nominatim(user_agent="location_mapping")


latitudes = []
longitudes = []

words_to_remove = ["Near", "Off"]
locations = [re.sub(rf'^{word}\s*', '', location) for word in words_to_remove for location in podatki["Location"]]

# Geocode the locations
for i,location in enumerate(locations[:3]):
    for word in words_to_remove:
        location = re.sub(rf'^{word}\s*', '', location)
    
    try:
        pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
        # Get the location information
        #location_info = geolocator.geocode(location)
        location_info  = pool.map(geocode_worker, location)
        # Append latitude and longitude to respective lists
        latitudes.append(location_info.latitude)
        longitudes.append(location_info.longitude)
    except Exception as e:
        print(f"Error geocoding {location}: {e}")
        
        
# Plot the locations on the map
plt.figure(figsize=(10, 6))
plt.scatter(longitudes, latitudes, color='red', marker='o')



# Add title and labels
plt.title('Locations Map')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Show the map
plt.grid(True)
plt.show()

